# Setup

In [1]:
import torch
import argparse
import numpy as np
import utils
import export

# Setup

parser = argparse.ArgumentParser()

parser.add_argument('--seed', type=int, default=123)
parser.add_argument('--method', type=str, default='SLL', choices=[
    'CER', 'REF', 'SLL', 'SLL_G'
])
parser.add_argument('--budget_pct', type=float, default=0.25)
parser.add_argument('--g0_method', type=str, default='random', choices=[
  'random', # randomly distribution of g0
  'large_cluster', # a random node and [g0_size] of its neighbors are in g0
  'many_clusters', # 10 random nodes and [g0_size] of their neighbors are in g0
  ])
parser.add_argument('--g0_size', type=float, default=0.2)
parser.add_argument('--lr', type=float, default=10)
parser.add_argument('--T_s', type=int, default=1263)
parser.add_argument('--T_u', type=int, default=-1)
parser.add_argument('--dataset', type=str, default='cora', choices=[
    'Cora', 'Cora-ML', 'Citeseer', 'Pubmed', 'Polblogs', 'ACM', 'BlogCatalog', 'Flickr', 'UAI'
])
parser.add_argument('--ptb_rate', type=float, default=0.25)
args = parser.parse_args("")

device = "cuda:1" if torch.cuda.is_available() else "cpu"
np.random.seed(args.seed)
torch.manual_seed(args.seed)

# Data


In [2]:
import random

def get_neighbor_subgraphs(adj: torch.Tensor, size: int, n: int) -> torch.Tensor:
    """
    Returns a tensor ~ `(n * size)` of node indices for `n` supgraphs of size `size`
    """
    res = torch.zeros([n, size], dtype=torch.long)

    for i in range(n):
        out = []
        stack = [random.randint(0, adj.shape[0] - 1)]
        while len(out) < size:
            if len(stack) == 0:
                stack.append(random.randint(0, adj.shape[0] - 1))
            curNode = stack.pop()
            if curNode not in out:
                out.append(curNode)
                children = adj[curNode].nonzero().t()[0].cpu().tolist()
                stack = children + stack
        res[i] = torch.tensor(out)

    return res

def get_rand_subgraphs(adj, size: int, n: int) -> torch.Tensor:
    return torch.randint(adj.shape[0], [n, size], dtype=torch.long)


# Data

In [3]:
num_samples = 1024
sample_size = 64
adj, feat, label, train_mask, val_mask, test_mask = utils.load_data(args.dataset, args.seed, device=device)

Loading cora dataset...
Selecting 1 largest connected components


In [4]:
def get_samples(A, num_samples, sample_size):
    s = get_neighbor_subgraphs(A.cpu(), sample_size, num_samples)

    temp = A[s]
    subgraph_feats = feat[s]
    subgraph_adjs = torch.zeros(num_samples, sample_size, sample_size).to(device)
    subgraph_labels = label[s]
    subgraph_train_masks = train_mask[s]
    for i in range(num_samples):
        subgraph_adjs[i] = temp[i][:,s[i]]

    return subgraph_feats, subgraph_adjs, subgraph_labels, subgraph_train_masks

In [5]:
subgraph_feats, subgraph_adjs, subgraph_labels, subgraph_train_masks = get_samples(adj, num_samples, sample_size)

# Methods

In [6]:
from torch_geometric.nn import Sequential, DenseGCNConv
from torch.nn import Linear, ReLU
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader

def get_gcn(hid: int=64):
    return Sequential('x, adj', [
        (DenseGCNConv(feat.shape[1], hid), 'x, adj -> x'),
        ReLU(inplace=True),
        (DenseGCNConv(hid, hid), 'x, adj -> x'),
        ReLU(inplace=True),
        Linear(hid, int(label.max()) + 1),
    ]).to(device)

def train(model, dataloader: DataLoader, epochs: int):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), 1e-3)
    t = tqdm(range(epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
    t.set_description("Model training")
    loss = torch.tensor(0)
    for _ in t:
        for feats, adjs, labels, train_masks in dataloader:
            pred = model(feats, adjs)
            mask = train_masks.flatten()
            loss = F.cross_entropy(pred.flatten(end_dim=1)[mask], labels.flatten(end_dim=1)[mask])
            loss.backward()
            optimizer.step()
            t.set_postfix({"loss": round(loss.item(), 2)})

def train_adj(model, feat, adj, label, train_mask, epochs: int):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), 1e-3)
    t = tqdm(range(epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
    t.set_description("Model training")
    loss = torch.tensor(0)
    for _ in t:
        pred = model(feat, adj)
        loss = F.cross_entropy(pred.squeeze()[train_mask], label[train_mask])
        loss.backward()
        optimizer.step()
        t.set_postfix({"loss": round(loss.item(), 2)})

def eval_adj(model, adj, test_mask):
    model.eval()
    pred = model(feat, adj)
    acc = ((pred.argmax(dim=2).squeeze() == label)[test_mask].sum() / test_mask.sum()).item()
    return acc

In [7]:
def get_dataloader(adj, num_samples, sample_size, batch_size):
    subgraph_feats, subgraph_adjs, subgraph_labels, subgraph_train_masks = get_samples(adj, num_samples, sample_size)
    return DataLoader(
    TensorDataset(subgraph_feats, subgraph_adjs, subgraph_labels, subgraph_train_masks), 
    batch_size=batch_size, shuffle=True)

gcn = get_gcn()
train(gcn, get_dataloader(adj, num_samples, sample_size, 128), 100)
eval_adj(gcn, adj, test_mask)

Model training: 100%|██████████| 100/100 [00:24<00:00,  4.03it/s, loss=0]  


Accuracy:  77.72%


In [44]:
from deeprobust.graph.defense import GCN
import deeprutils
# ATTACK

def PGD_Attack(A, X, ε, T, train_mask, val_mask, surrogate_lr=1e-2, epochs=30):
    # A = adj
    # ε = int(0.5 * adj.sum().item())
    # X = feat
    # T = label
    # surrogate_lr = 1e-2
    # epochs = 30

    M = torch.zeros_like(A).float().to(device)
    θ = GCN(nfeat=X.shape[1], nclass=T.max().item()+1, nhid=32, lr=surrogate_lr, device=device).to(device)
    # θ.fit(X, A, T, train_mask, val_mask, train_iters=100)

    t = tqdm(range(epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
    A_p = torch.zeros_like(A).to(device).requires_grad_(True) # Initialize modified adj

    for epoch in t:
        pred = θ(X, A_p)
        L = F.cross_entropy(pred.squeeze(), T)
        A_grad = torch.autograd.grad(L, A_p)[0]
        lr = 1 / np.sqrt(epoch + 1)
        M.data.add_(lr * A_grad)
        M = utils.truncate(M, A)
        M = utils.projection(M, ε)
        A_p = utils.xor(A, utils.discretize(M)).to(device).requires_grad_(True)
        θ.fit(X, A_p, T, train_mask, val_mask, train_iters=1)

        t.set_postfix({
            "loss": L.item(),
        })

    best = torch.tensor([0])
    max_loss = -10000
    for i in range(10):
        A_p = utils.xor(A, utils.discretize(M)).to(device).requires_grad_(True)
        pred = θ(X, A_p)
        L = F.cross_entropy(pred.squeeze(), T)

        if L.item() > max_loss:
            max_loss = L.item()
            best = A_p

    best.requires_grad_(False)
    return best

In [45]:
def Partition_PGD(k, A, X, ε, T, train_mask, val_mask, surrogate_lr=1e-2, epochs=30):
    size = int(A.shape[0] / k)

    batched_A_p = A.clone()

    for i in range(k):
        sub_A = A[size * i : size * (i + 1)][:,size * i : size * (i + 1)]
        sub_X = X[size * i : size * (i + 1)]
        sub_T = T[size * i : size * (i + 1)]
        sub_tmask = train_mask[size * i : size * (i + 1)]
        sub_vmask = val_mask[size * i : size * (i + 1)]

        sub_best = PGD_Attack(sub_A, sub_X, ε / k, sub_T, sub_tmask, sub_vmask, surrogate_lr=surrogate_lr, epochs=epochs)

        batched_A_p[size * i : size * (i + 1)][:,size * i : size * (i + 1)] = sub_best
    
    return batched_A_p


In [46]:
def Batch_PGD(size, n, A, X, ε, T, train_mask, val_mask, surrogate_lr=1e-2, epochs=30):
    batched_A_p = A.clone()

    for i in range(n):
        indices = torch.randperm(A.shape[0])[:size]
        sub_A = A[indices][:,indices]
        sub_X = X[indices]
        sub_T = T[indices]
        sub_tmask = train_mask[indices]
        sub_vmask = val_mask[indices]

        sub_best = PGD_Attack(sub_A, sub_X, ε / n, sub_T, sub_tmask, sub_vmask, surrogate_lr=surrogate_lr, epochs=epochs)

        batched_A_p[indices][:,indices] = sub_best
    
    return batched_A_p


In [ ]:
budget = int(0.5 * adj.sum().item())
PGD_Attack(adj, feat, budget, label, train_mask, val_mask)
test = get_gcn()
train_adj(test, feat, batched_A_p, label, train_mask, 100)
eval_adj(test, batched_A_p, test_mask)


b_best = Batch_PGD(sample_size, num_samples, adj, feat, budget, label, train_mask, val_mask)

Model training: 100%|██████████| 100/100 [00:02<00:00, 35.40it/s, loss=0.97]


Accuracy:  45.67%


In [42]:
gcn = get_gcn()
train(gcn, get_dataloader(best, num_samples, sample_size, 128), 100)
eval_adj(gcn, best, test_mask)

Model training: 100%|██████████| 100/100 [00:18<00:00,  5.50it/s, loss=0]


Accuracy:  67.15%
